<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Gemma_7B_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Necessary Dependencies:**

In [ ]:
!pip install -qU bitsandbytes
!pip install -qU trl
!pip install -qU transformers
!pip install -qU peft
!pip install -qU optimum
!pip install -qU datasets
!pip install -qU accelerate
!pip install -qU nltk
!pip install -qU rouge_score

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
dataset = load_dataset("kaifahmad/indian-history-hindi-QA-3.4k",split="train")

In [ ]:
dataset

Dataset({
    features: ['Question', 'Answer'],
    num_rows: 3468
})

In [ ]:
def process_data_sample(example):
    # Extract relevant information from the example
    question = example['Question']
    answer = example['Answer']
    # Prepare the processed example for a History Question Answering System
    processed_example = (
        "आप एक इतिहास प्रश्न-उत्तर प्रणाली हैं, जो इतिहास से संबंधित ज्ञान और पूछे जाने वाले प्रश्नों को संग्रहीत करती है। "
        "आप इतिहास से संबंधित तथ्य प्रदान कर सकते हैं, प्रश्नों का उत्तर दे सकते हैं और इतिहास संबंधी मार्गदर्शन प्रदान कर सकते हैं।\n\n"
        f"उपयोगकर्ता प्रश्न:\n{question}\n\n"
        f"उत्तर:\n{answer}\n\n"
    )
    return processed_example

In [ ]:
dataset = dataset.map(lambda example: {'text': process_data_sample(example)}, remove_columns=['Question', 'Answer'])

In [ ]:
for i in range(5):
    print(dataset[i]['text'])

आप एक इतिहास प्रश्न-उत्तर प्रणाली हैं, जो इतिहास से संबंधित ज्ञान और पूछे जाने वाले प्रश्नों को संग्रहीत करती है। आप इतिहास से संबंधित तथ्य प्रदान कर सकते हैं, प्रश्नों का उत्तर दे सकते हैं और इतिहास संबंधी मार्गदर्शन प्रदान कर सकते हैं।

उपयोगकर्ता प्रश्न:
प्राचीन मानवों के भारतीय उपमहाद्वीप में प्रवास के प्रमुख सम्भावित मार्ग क्या थे?

उत्तर:
प्राचीन मानवों का भारतीय उपमहाद्वीप में प्रवास संभवतः अरब सागर और बंगाल की खाड़ी के समुद्रतटीय मार्गों के साथ ही उत्तर-पश्चिमी प्रवासी द्वार के माध्यम से भी हो सकता था।


आप एक इतिहास प्रश्न-उत्तर प्रणाली हैं, जो इतिहास से संबंधित ज्ञान और पूछे जाने वाले प्रश्नों को संग्रहीत करती है। आप इतिहास से संबंधित तथ्य प्रदान कर सकते हैं, प्रश्नों का उत्तर दे सकते हैं और इतिहास संबंधी मार्गदर्शन प्रदान कर सकते हैं।

उपयोगकर्ता प्रश्न:
प्राचीन मानवों को भारतीय उपमहाद्वीप में प्रवास करने के लिए कौन-कौन से कारक प्रेरित कर सकते थे?

उत्तर:
प्राचीन मानवों का भारतीय उपमहाद्वीप में प्रवास विभिन्न कारकों से प्रेरित हो सकता था, जैसे कि पर्यावरणीय परिवर्तन, नए संसा

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    pipeline,
    logging
)
from sklearn.model_selection import train_test_split
from datasets import Dataset
from huggingface_hub import notebook_login
from google.colab import drive
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

**Model Loading and Quantization:**

In [ ]:
model_name = 'google/gemma-7b'

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Tokenization and Padding
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA Config
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias='none',
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

In [ ]:
# Training Arguments
training_arguments = TrainingArguments(
    output_dir='Finetuned-Model',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim='paged_adamw_32bit',
    learning_rate=2e-4,
    lr_scheduler_type='cosine',
    save_strategy='epoch',
    logging_steps=10,
    save_steps=10,
    num_train_epochs=3,
    max_steps=200,
    fp16=True,
    warmup_ratio=0.05,
    push_to_hub=False,
)

In [ ]:
# SFT Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field='text',
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=264
)

AttributeError: 'dict' object has no attribute 'column_names'